<a href="https://colab.research.google.com/github/JAILBREAK-101/Google_Colab_Project/blob/main/language_translator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Necessary libraries
pip install "tensorflow-text>=2.11"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [2]:
pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00


In [4]:
# Install the necessary Language Translation libraries
import numpy as np

import typing
from typing import Any, Tuple

import tensorflow as tf
import tensorflow_text as tf_text

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [5]:
# Download the file
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'fra-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'fra-eng/fra.txt'

3423204/3423204 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [20]:
# Load the dataset and clean it
def load_data(path):
  """
  Loads target and context sentences from a tab-delimited file.

  Args:
      path: Path to the file containing parallel sentences.

  Returns:
      A tuple containing two NumPy arrays: target sentences and context sentences.
  """

  text = path.read_text(encoding='utf-8')
  lines = text.splitlines()

  # Check if each line has exactly two elements after splitting
  filtered_pairs = [line.split('\t') for line in lines if len(line.split('\t')) == 2]

  # Unpack only the first two elements (assuming target is first)
  target = np.array([pair[0] for pair in filtered_pairs])
  context = np.array([pair[1] for pair in filtered_pairs])

  return target, context

target_sentences, context_sentences = load_data(pathlib.Path("/content/fra.txt")) # Changed path to correct location
print("Target sentences:\n", target_sentences[:5])  # Print only the first 5 target sentences
print("Context sentences:\n", context_sentences[:5])  # Print only the first 5 context sen

Target sentences:
 []
Context sentences:
 []


In [23]:
# Create the tf.data dataset: shuffle and barch them efficiently
BUFFER_SIZE = len(context_sentences) # Make sure BUFFER_SIZE is set correctly
BATCH_SIZE = 64

dataset_is_train = np.random.uniform(size=(len(target_sentences),)) < 0.8

# Fix: Ensure BUFFER_SIZE is greater than 0
if BUFFER_SIZE == 0:
  BUFFER_SIZE = 1  # Set a minimum buffer size if the dataset is empty

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_sentences[dataset_is_train], target_sentences[dataset_is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((context_sentences[~dataset_is_train], target_sentences[~dataset_is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

In [24]:
# Example of using our dataset

for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])
  break

In [25]:
# TEXT PREPROCESSING
# All the text processing happens inside the model
# standardize the input text.
# Unicode normalization to split accented characters and replace compatibility characters with their ASCII equivalents.

# Punctuation removal
def tf_lower_and_split_punct(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [31]:
# Testing text processing
sample_french_text = tf.constant("Est-il toujours à la maison?")

print(sample_french_text.numpy())
print(tf_text.normalize_utf8(sample_french_text, 'NFKD').numpy())

print(sample_french_text.numpy().decode())
print(tf_lower_and_split_punct(sample_french_text).numpy().decode())

b'Est-il toujours \xc3\xa0 la maison?'
b'Est-il toujours a\xcc\x80 la maison?'
Est-il toujours à la maison?
[START] estil toujours a la maison ? [END]


In [32]:
# TEXT VECTORIZATION: Conversion from text to numerical representation

max_vocab_size = 5000

context_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

In [33]:
# This method reads one epoch of the training data, and works a lot like Model.fit
context_text_processor.adapt(train_raw.map(lambda context, target: context))

# Here are the first 10 words from the vocabulary:
context_text_processor.get_vocabulary()[:10]

# This is the French Text Vectorization layer above

['', '[UNK]']

In [34]:
# ENGLISH VECTORIZATION LAYER
target_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size,
    ragged=True)

target_text_processor.adapt(train_raw.map(lambda context, target: target))
target_text_processor.get_vocabulary()[:10]

['', '[UNK]']

In [36]:
# these layers can convert a batch of strings into a batch of token IDs
example_context_strings = ["example sentence 1", "another example"] # Define the variable with example strings.
example_batch_tokens = context_text_processor(example_context_strings)
example_batch_tokens[:3, :]

FailedPreconditionError: Exception encountered when calling TextVectorization.call().

[1m{{function_node __wrapped__LookupTableFindV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Table not initialized. [Op:LookupTableFindV2] name: [0m

Arguments received by TextVectorization.call():
  • inputs=["'example sentence 1'", "'another example'"]

In [ ]:
# convert token IDs back to text
context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens[0].numpy()]
' '.join(tokens)